In [ ]:
import pandas as pd
import numpy as np
import torch
from torchvision import datasets,transforms,models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import random
import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error as rmse
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import albumentations as A
import tqdm
from tqdm import trange,tqdm
from time import sleep
from sklearn.model_selection import train_test_split
!pip install timm
!pip install torchinfo
from torchinfo import summary
import timm

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
tidcol = test['Id']
idcol = train['Id']
direct1 = '../input/petfinder-pawpularity-score/train'
for i in range(idcol.shape[0]):
    idcol[i] = os.path.join(direct1, f'{idcol[i]}.jpg')
direct = '../input/petfinder-pawpularity-score/test'
for i in range(tidcol.shape[0]):
    tidcol[i] = os.path.join(direct, f'{tidcol[i]}.jpg')

In [ ]:
#  P<12.5 = 0
# 12.5 <= P < 37.5 = 1
# 37.5<=P <  62.5 = 2
# 62.5<P < 87.5  = 3
# P>=87.5 = 4
Y = []
for element in train['Pawpularity']:
    if element < 12.5:
        Y.append(0)
    elif 12.5<= element < 37.5 :
        Y.append(1)
    elif 37.5<= element < 62.5 :
        Y.append(2)
    elif 62.5<= element < 87.5 :
        Y.append(3)
    else:
        Y.append(4)
Y = np.array(Y)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train['Id'].values,Y,test_size = 0.15,random_state = 69)

In [ ]:
class Pawpularity(Dataset):
    def __init__(self,path,targets=None,transform = None):
        self.path = path
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self,idx):
        ipath = self.path[idx]
        image = Image.open(ipath)
        #image = cv2.imread(ipath)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #conv = transforms.ToTensor()
        #image = conv(image)
        #image = torch.rot90(image,1,[1,2])
        image = np.array(image)
        if self.transform is not None:
            al_dict = self.transform(image=image)
            image = al_dict['image']
        #print(image.size)
        
        conv = transforms.ToTensor()
        image = conv(image)
        image = torch.rot90(image,1,[1,2])
        if self.targets is not None :
            label = torch.tensor(self.targets[idx]).long()
            return image,label
        else :
            return image

In [ ]:
transformT = A.Compose([
    A.SmallestMaxSize(240),
    A.RandomCrop(width=224 , height=224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.Normalize(mean=(0.518, 0.484, 0.446), std=(0.264, 0.260, 0.261))
])
transformV = A.Compose([

    A.Resize(width=224 , height=224),
    A.Normalize(mean=(0.518, 0.484, 0.446), std=(0.264, 0.260, 0.261))
])

In [ ]:
traindata = Pawpularity(path  = X_train,targets = y_train , transform = transformT)
valdata = Pawpularity(path = X_val,targets = y_val , transform = transformV)

In [ ]:
def imageplot(dataset , count =3):
    plt.figure(figsize = (20,20))
    for i in range(count):
        rand = random.randint(0, len(dataset))
        #print(len(dataset))
        image,label = dataset[rand]
        plt.subplot(1, count, i%count +1)
        plt.axis('off')
        plt.imshow(image.permute(2, 1, 0))
        plt.title(f'Pawpularity: {label}')

In [ ]:
for i in range(4):
    imageplot(traindata,count=4)

In [ ]:
train_loader = DataLoader(dataset=traindata, 
                          batch_size=32, 
                          shuffle=True)

val_loader = DataLoader(dataset=valdata, 
                         batch_size=32, 
                         shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if(torch.cuda.is_available()):
  print("The device is " + torch.cuda.get_device_name())
else:
    print("Using cpu no gpu available")
print('\n',device)

In [ ]:
all_densenet_models = timm.list_models('*resnet*')
all_densenet_models

In [ ]:
r50 = timm.create_model('resnet50', pretrained=True)
summary(r50, input_size=(16, 3, 224,224))

In [ ]:
in_channels = r50.fc.in_features
cherry = nn.Sequential(
    nn.Linear(in_channels,1000),
    nn.ReLU(inplace = True),
    nn.BatchNorm1d(1000),
    nn.Dropout(0.5),
    nn.Linear(1000,5)
    )
r50.fc = cherry

In [ ]:
metric = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(r50.parameters(),lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 5,
                                                        gamma = 0.314,verbose = True)


In [ ]:
def trainer(model,optimizer,lossfn,scheduler,epochs,device,dataloader,val_loader=None):
    model.to(device)
    for epoch in range(epochs):
                print(f'Starting epoch {epoch+1}')
                current_loss = 0.0
                model = model.train()
                scores = []
                targets = []
                with tqdm(dataloader, unit="batch") as tepoch:
                    for images,target in tepoch:
                        tepoch.set_description(f"Epoch {epoch}")
                        images = images.to(device)
                        target = target.to(device)
                        score = model(images)
                        #print(score.shape,target.shape)
                        a,b = score.shape
                        target = torch.reshape(target,(a,))
                        loss = lossfn(score,target)
                        targets.append(target.detach().cpu().numpy())
                        scores.append(score.detach().cpu().numpy())
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        tepoch.set_postfix(loss=loss.item())
                        sleep(0.1)
                #val_score = validate(model,val_loader)
                #vrmse.append(val_score)
                ep.append(epoch+1)
                #train_score = rmse(np.concatenate(scores,axis= 0),np.concatenate(targets,axis = 0),squared = False)
                #trmse.append(train_score)
                #print(str(epoch+1) + ": validation rmse : " + str(val_score))
                #print(str(epoch +1) + " : traiin rmse :" + str(train_score) )
                scheduler.step()
                print(optimizer.state_dict()['param_groups'][0]['lr'])
                #torch.save(model,'/kaggle/working/model2.pth')   

In [ ]:
trainer(r50,optimizer,metric,scheduler,2,device,train_loader)